In [1]:
# Daniel Bandala @ sep 2022
import cv2 as cv
import numpy as np

# Optical flow - Farneback algorithm (Opencv)
Optical flow or optic flow is the pattern of apparent motion of objects, surfaces, and edges in a visual scene caused by the relative motion between an observer and a scene. Optical flow can also be defined as the distribution of apparent velocities of movement of brightness pattern in an image. It is known as the pattern of apparent motion of objects, i.e, it is the motion of objects between every two consecutive frames of the sequence, which is caused by the movement of the object being captured or the camera capturing it. Consider an object with intensity $I (x, y, t)$, after time dt, it moves to by dx and dy, now, the new intensity would be, $I (x+dx, y+dy, t+dt)$.

In dense optical flow, we look at all of the points (unlike Lucas Kanade which works only on corner points detected by Shi-Tomasi Algorithm) and detect the pixel intensity changes between the two frames, resulting in an image with highlighted pixels, after converting to hsv format for clear visibility. It computes the magnitude and direction of optical flow from an array of the flow vectors, i.e., $(dx/dt, dy/dt)$. Later it visualizes the angle or direction of flow by hue and the distance or magnitude of flow by value of HSV color representation. For visibility to be optimal, strength of HSV is set to 255. OpenCV provides a function cv2.calcOpticalFlowFarneback to look into dense optical flow.


In [2]:
def calc_farneback(frames):
    try:
        return cv.calcOpticalFlowFarneback(
            frames[0], frames[1],
            # options, defaults
            None,  # output
            0.5,  # pyr_scale, 0.5
            10,  # levels, 3
            min(frames[0].shape[:2]) // 5,  # winsize, 15
            10,  # iterations, 3
            7,  # poly_n, 5
            1.5,  # poly_sigma, 1.2
            cv.OPTFLOW_FARNEBACK_GAUSSIAN,  # flags, 0
        )
    except cv2.error:
        return None 

In [3]:
cap = cv.VideoCapture(1)
ret, frame1 = cap.read()

[ WARN:0@1.768] global /home/bandala/opencv/modules/videoio/src/cap_gstreamer.cpp (1412) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


In [4]:
prvs = cv.cvtColor(frame1,cv.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[...,1] = 255

In [5]:
while(1):
    ret, frame2 = cap.read()    
    # Our operations on the frame come here
    next_f = cv.cvtColor(frame2,cv.COLOR_BGR2GRAY)
    flow = calc_farneback([prvs,next_f])
    mag, ang = cv.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv.normalize(mag,None,0,255,cv.NORM_MINMAX)
    bgr = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)
    prvs = next_f
    # Display the resulting frame
    cv.imshow('Optical Flow Aura',bgr)
    if cv.waitKey(2) & 0xFF == ord('q'):  # press q to quit
        break
# When everything done, release the capture
cap.release()
cv.destroyAllWindows()